[View in Colaboratory](https://colab.research.google.com/github/nautilusPrime/cifar10-image-classifier/blob/master/cifar10.ipynb)

In [53]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
import os

In [54]:
batch_size = 32
num_classes = 10
epochs = 100
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [55]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [56]:
l2_lambda = 0.0001

In [58]:
# You can modify Model here.
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:],
                 kernel_initializer='he_uniform', 
                 kernel_regularizer=l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3),
                 kernel_initializer='he_uniform', 
                 kernel_regularizer=l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same',
                 kernel_initializer='he_uniform', 
                 kernel_regularizer=l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3),
                 kernel_initializer='he_uniform', 
                 kernel_regularizer=l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,
               kernel_initializer='he_uniform', 
               kernel_regularizer=l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(num_classes,
                kernel_initializer='glorot_uniform', 
                kernel_regularizer=l2(l2_lambda)))
model.add(Activation('softmax'))

In [59]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])

In [60]:
x_train = x_train.astype('float16')
x_test = x_test.astype('float16')
x_train /= 255
x_test /= 255

In [61]:
data_augmentation = False

In [62]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=[EarlyStopping(monitor='val_loss', patience=5)])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), samples_per_epoch=len(x_train),
                        epochs=epochs, validation_data=(x_test, y_test), workers=4)

Not using data augmentation.
Train on 50000 samples, validate on 10000 samples
Epoch 1/100
36448/50000 [====================>.........] - ETA: 16s - loss: 1.7405 - acc: 0.4805

50000/50000 [==============================] - 62s 1ms/step - loss: 1.6407 - acc: 0.5169 - val_loss: 1.7037 - val_acc: 0.5170
Epoch 2/100
23648/50000 [=============>................] - ETA: 30s - loss: 1.3278 - acc: 0.6399

50000/50000 [==============================] - 60s 1ms/step - loss: 1.3179 - acc: 0.6503 - val_loss: 1.5249 - val_acc: 0.6032
Epoch 3/100
18784/50000 [==========>...................] - ETA: 36s - loss: 1.2817 - acc: 0.6846

50000/50000 [==============================] - 60s 1ms/step - loss: 1.2979 - acc: 0.6900 - val_loss: 1.2210 - val_acc: 0.7321
Epoch 4/100
16896/50000 [=========>....................] - ETA: 37s - loss: 1.2852 - acc: 0.7108

50000/50000 [==============================] - 60s 1ms/step - loss: 1.2829 - acc: 0.7155 - val_loss: 1.1991 - val_acc: 0.7459
Epoch 5/100
16224/50000 [========>.....................] - ETA: 38s - loss: 1.2445 - acc: 0.7329

...

50000/50000 [==============================] - 60s 1ms/step - loss: 1.0636 - acc: 0.7940 - val_loss: 1.0329 - val_acc: 0.8083
Epoch 28/100
15712/50000 [========>.....................] - ETA: 39s - loss: 1.0257 - acc: 0.8064

50000/50000 [==============================] - 61s 1ms/step - loss: 1.0561 - acc: 0.8004 - val_loss: 1.1516 - val_acc: 0.7758
Epoch 29/100
15712/50000 [========>.....................] - ETA: 39s - loss: 1.0193 - acc: 0.8107

50000/50000 [==============================] - 60s 1ms/step - loss: 1.0484 - acc: 0.8004 - val_loss: 1.0240 - val_acc: 0.8076
Epoch 30/100
15712/50000 [========>.....................] - ETA: 39s - loss: 1.0341 - acc: 0.8044

50000/50000 [==============================] - 60s 1ms/step - loss: 1.0561 - acc: 0.7975 - val_loss: 1.0379 - val_acc: 0.8038
Epoch 31/100
15488/50000 [========>.....................] - ETA: 40s - loss: 1.0162 - acc: 0.8093

50000/50000 [==============================] - 61s 1ms/step - loss: 1.0495 - acc: 0.7991 - val_loss: 1.0483 - val_acc: 0.8061


In [0]:
from google.colab import files 

In [0]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /content/saved_models/keras_cifar10_trained_model.h5 


In [0]:
files.download(model_path)

In [11]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 3s 284us/step
Test loss: 0.6678660587310791
Test accuracy: 0.8141
